In [1]:
import os

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [5]:
%pwd

'/Users/badaradiallo/Desktop/DataScience_Projetc/research'

In [6]:
os.chdir('../')

In [7]:
%pwd

'/Users/badaradiallo/Desktop/DataScience_Projetc'

In [9]:
from src.datascience.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from src.datascience.utils.common import read_yaml,create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH
                 ,schema_filepath = SCHEMA_FILE_PATH): 
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        ) 
        
        return data_ingestion_config

In [11]:
## Data Ingestion Component
import urllib.request as request
from src.datascience import logger
import zipfile
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download with the following info: \n{headers}")
        else:
            logger.info("File already exist")
            
    def extract_zipfile(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

[2025-06-20 20:22:58,689: INFO: common: yaml file: /Users/badaradiallo/Desktop/DataScience_Projetc/config/config.yaml loaded successfully]
[2025-06-20 20:22:58,691: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-20 20:22:58,691: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-20 20:22:58,693: INFO: common: created directory at: artifacts]
[2025-06-20 20:22:58,695: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-20 20:22:59,286: INFO: 2304925066: artifacts/data_ingestion/data.zip download with the following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9CEB:DD277:3A7831: